<a href="https://colab.research.google.com/github/kanade2001/KokomeloTalk/blob/g0roro%2Festimate_emotion/emotion_estimate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 必要なモジュールのインストール/インポート
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
! pip install fugashi ipadic
! pip install unidic_lite
import numpy as np

  Using cached fugashi-1.4.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.9 kB)
  Using cached ipadic-1.0.0.tar.gz (13.4 MB)
  Preparing metadata (setup.py) ... done
Using cached fugashi-1.4.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (671 kB)
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=3ab7a1771b29148478a84434bc3051c5d086498da75f9452115b20163cd211a8
  Stored in directory: /root/.cache/pip/wheels/5b/ea/e3/2f6e0860a327daba3b030853fce4483ed37468bbf1101c59c3
Successfully built ipadic
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic_lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=3ab2c59a6c06d367053e4378b6673f941efc3f5c1398e9821a71d4363fa18fd5
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic_lite


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
load_directory = '/content/drive/My Drive/trained_model'

# モデルとトークナイザを読み込み
from transformers import AutoModelForSequenceClassification, AutoTokenizer
loaded_model = AutoModelForSequenceClassification.from_pretrained(load_directory)
loaded_tokenizer = AutoTokenizer.from_pretrained(load_directory)

# デバイスを指定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# 感情名（日本語）を定義
emotion_names_jp = ['喜び', '悲しみ', '怒り', '恐れ', '嫌悪', '驚き', '信頼', '期待'] # 例として8つの感情を想定


# ソフトマックス関数
# https://www.delftstack.com/ja/howto/numpy/numpy-softmax/
def np_softmax(x):
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x

def analyze_emotion(text, show_fig=False):
    # 推論モードを有効化
    loaded_model.eval()

    # 入力データ変換 + 推論
    tokens = loaded_tokenizer(text, truncation=True, return_tensors="pt")
    tokens.to(loaded_model.device)
    preds = loaded_model(**tokens)
    prob = np_softmax(preds.logits.cpu().detach().numpy()[0])
    out_dict = {n: p for n, p in zip(emotion_names_jp, prob)}
    out_dict_ud = sorted(out_dict.items(), key=lambda x:x[1], reverse=True)


    # 棒グラフを描画
    if show_fig:
        plt.figure(figsize=(8, 3))
        df = pd.DataFrame(out_dict.items(), columns=['name', 'prob'])
        sns.barplot(x='name', y='prob', data=df)
        plt.title('入力文 : ' + text, fontsize=15)
    else:
        print(out_dict_ud)

analyze_emotion('昨日彼女に振られました')

[('悲しみ', 0.9329988), ('信頼', 0.039915565), ('驚き', 0.01132242), ('恐れ', 0.00851324), ('喜び', 0.003976579), ('嫌悪', 0.002052537), ('怒り', 0.0010010423), ('期待', 0.0002198929)]
